In [14]:
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format ,from_unixtime ,monotonically_increasing_id
from pyspark.sql import types as t
from pyspark.sql import SparkSession, SQLContext, GroupedData
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .enableHiveSupport().getOrCreate()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
# read  PORT statstics data
df = spark.read.format("parquet").load("s3n://capstone-rk/visitors_per_ports-per-port")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1568404209694_0022,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+----+-----+
|                port|visitors|year|month|
+--------------------+--------+----+-----+
|MOSES LAKE GRANT ...|       3|2016|    7|
|HULL FIELD, SUGAR...|      33|2016|    6|
|MC GUIRE AFB - WR...|       3|2016|    7|
|CANNON INTL - REN...|     551|2016|    7|
|GALLATIN FIELD - ...|      20|2016|    7|
|MID-CONTINENT - W...|       1|2016|    7|
|OAKLAND COUNTY - ...|       9|2016|    7|
|CANNON INTL - REN...|     577|2016|    6|
|MID-CONTINENT - W...|      15|2016|    6|
|OAKLAND COUNTY - ...|       9|2016|    6|
|HANSCOM FIELD - B...|      31|2016|    6|
|BLUE GRASS - LEXI...|       1|2016|    7|
|THOUSAND ISLAND B...|    3347|2016|    7|
|LAREDO COLUMBIA B...|     121|2016|    6|
|THOUSAND ISLAND B...|    1665|2016|    6|
|BORINQUEN - AGUAD...|      50|2016|    7|
|JUAREZ-LINCOLN BR...|    1178|2016|    7|
|STEWART - ORANGE ...|       7|2016|    7|
|STEWART - ORANGE ...|       4|2016|    6|
|BRADENTON - SARAS...|      27|2016|    7|
+----------

In [3]:
# PIVOT the DATA by month
# We can do this by Year too. Since we have loaded data for only one year, we are pivoting by just month
df2 = df.groupby("port","Year").pivot("month").avg('visitors')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+----+--------+--------+
|                port|Year|null|       6|       7|
+--------------------+----+----+--------+--------+
|FAJARDO, PR      ...|2016|null|     8.0|    null|
|MINOT, ND        ...|2016|null|     6.0|     3.0|
|NASHVILLE, TN    ...|2016|null|   167.0|   317.0|
|MORGAN, MT       ...|2016|null|     3.0|     5.0|
|CINCINNATI, OH   ...|2016|null|  1593.0|  1737.0|
|MIAMI, FL        ...|2016|null|387879.0|489832.0|
|NORTHGATE, ND    ...|2016|null|    10.0|    15.0|
|RICHMOND, VA     ...|2016|null|    null|    45.0|
|HARLINGEN, TX    ...|2016|null|    14.0|    14.0|
|SANTA ANA, CA    ...|2016|null|    93.0|   187.0|
|PEACE BRIDGE, NY ...|2016|null|  6960.0| 14151.0|
|SUMAS, WA        ...|2016|null|  1199.0|  2287.0|
|HANSCOM FIELD - B...|2016|null|    31.0|    43.0|
|WILLISTON, ND    ...|2016|null|     5.0|    null|
|LONG BEACH, CA   ...|2016|null|   176.0|   105.0|
|MILWAUKEE, WI    ...|2016|null|    90.0|   199.0|
|SANFORD, FL      ...|2016|null

In [26]:

# add alias to column name
df3=df2.select(df2.port.alias("Port"), df2.Year.alias("Year") , df2["6"].alias("June") , df2["7"].alias("July")  )
df5 =df3.fillna(0)
df5.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+--------+--------+
|                Port|Year|    June|    July|
+--------------------+----+--------+--------+
|FAJARDO, PR      ...|2016|     8.0|     0.0|
|MINOT, ND        ...|2016|     6.0|     3.0|
|NASHVILLE, TN    ...|2016|   167.0|   317.0|
|MORGAN, MT       ...|2016|     3.0|     5.0|
|MIAMI, FL        ...|2016|387879.0|489832.0|
|CINCINNATI, OH   ...|2016|  1593.0|  1737.0|
|NORTHGATE, ND    ...|2016|    10.0|    15.0|
|RICHMOND, VA     ...|2016|     0.0|    45.0|
|SUMAS, WA        ...|2016|  1199.0|  2287.0|
|SANTA ANA, CA    ...|2016|    93.0|   187.0|
+--------------------+----+--------+--------+
only showing top 10 rows

In [27]:
# compute % change
df4 = df5.withColumn("% change in July" , format_number( ((( df5["July"] - df5["June"]  ) /  df5["June"] ) * 100),2))
#df4 = df3.withColumn("% change in July" ,   ((( df3["July"] - df3["June"] )  /  df3["June"] ) * 100))
          

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
df4.select ("Port", "year",format_number("June",0).alias("June") ,format_number("July",0).alias("July") , "% change in July"  ).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-------+-------+----------------+
|                Port|year|   June|   July|% change in July|
+--------------------+----+-------+-------+----------------+
|FAJARDO, PR      ...|2016|      8|      0|         -100.00|
|MINOT, ND        ...|2016|      6|      3|          -50.00|
|NASHVILLE, TN    ...|2016|    167|    317|           89.82|
|MORGAN, MT       ...|2016|      3|      5|           66.67|
|CINCINNATI, OH   ...|2016|  1,593|  1,737|            9.04|
|MIAMI, FL        ...|2016|387,879|489,832|           26.28|
|NORTHGATE, ND    ...|2016|     10|     15|           50.00|
|RICHMOND, VA     ...|2016|      0|     45|            null|
|HARLINGEN, TX    ...|2016|     14|     14|            0.00|
|SANTA ANA, CA    ...|2016|     93|    187|          101.08|
|HANSCOM FIELD - B...|2016|     31|     43|           38.71|
|PEACE BRIDGE, NY ...|2016|  6,960| 14,151|          103.32|
|SUMAS, WA        ...|2016|  1,199|  2,287|           90.74|
|WILLISTON, ND    ...|20